In [62]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from datasets import load_dataset
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from pytorch_fid import fid_score
from torchvision.utils import save_image
import torch.nn.functional as F
import torchvision.utils as vutils
import random

In [63]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from datasets import load_dataset
from PIL import Image
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# File paths
dataset_filepath = r"/home/sriyar/ADRL_Assignment2/butterflies_data/2/Training_set.csv"
dataset_filepath_test = r"/home/sriyar/ADRL_Assignment2/butterflies_data/2/Testing_set.csv"

# Define image transformations
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load datasets
dataset_train = load_dataset('csv', data_files=dataset_filepath)

# Create a mapping from labels to indices
label_to_idx = {label: idx for idx, label in enumerate(set(dataset_train['train']['label']))}

# Function to transform images and labels for training dataset
def transform_image_train(data):
    image = Image.open('/home/sriyar/ADRL_Assignment2/butterflies_data/2/train/' + data['filename'])
    data['image'] = data_transforms(image)
    data['label'] = label_to_idx[data['label']]  # Convert string label to index
    return data

# Apply transformations to the datasets
dataset_train = dataset_train.map(transform_image_train)

# Set format for PyTorch tensors
dataset_train.set_format(type='torch', columns=['image', 'label'])

# Split the training dataset into train, validation, and test sets
train_size = int(0.98 * len(dataset_train['train']))  
val_size = int(0.01 * len(dataset_train['train']))   
test_size = len(dataset_train['train']) - train_size - val_size  
train_dataset, val_dataset, test_dataset = random_split(dataset_train['train'], [train_size, val_size, test_size])

# Create DataLoaders
dataloader_butterfly_train = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=5)
dataloader_butterfly_val = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=5)
dataloader_butterfly_test = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=5)

# Count unique classes in the training dataset
num_classes = len(label_to_idx)  # Unique class count from the mapping
print(f"Number of classes: {num_classes}")


Number of classes: 75


In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Adjusted for input size (128x128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)  # Adding dropout

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        return x

# Create model instance
model = CNNClassifier(num_classes=num_classes).to(device)


# Training Loop with Early Stopping
num_epochs = 200  # Adjust as needed
best_val_loss = float('inf')
patience = 5  # Number of epochs with no improvement after which training will be stopped
counter = 0
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for batch in dataloader_butterfly_train:
        images, labels = batch['image'].to(device), batch['label'].to(device)
        optimizer.zero_grad()  # Zero the gradients

        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()  # Accumulate loss

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(dataloader_butterfly_train):.4f}")

    # Validation Step
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader_butterfly_val:
            images, labels = batch['image'].to(device), batch['label'].to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)  # Validation loss
            val_loss += loss.item()  # Accumulate validation loss
            _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(dataloader_butterfly_val)
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy on val data: {100 * correct / total:.2f}%")

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model.pth')  # Save the model
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

# Evaluate on the test set
model.load_state_dict(torch.load('best_model.pth'))  # Load the best model
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for batch in dataloader_butterfly_test:
        images, labels = batch['image'].to(device), batch['label'].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Accuracy on test data: {100 * correct / total:.2f}%")


In [1]:
import torch
import pickle
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from datasets import load_dataset
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from pytorch_fid import fid_score
from torchvision.utils import save_image
import torch.nn.functional as F
import torchvision.utils as vutils
import random
# Load the VAE model


In [2]:


import torch
from torch import nn
from torch.nn import functional as F


class VAE(nn.Module):
    def __init__(self, zsize, layer_count=3, channels=3):
        super(VAE, self).__init__()

        d = 128
        self.d = d
        self.zsize = zsize

        self.layer_count = layer_count

        mul = 1
        inputs = channels
        for i in range(self.layer_count):
            setattr(self, "conv%d" % (i + 1), nn.Conv2d(inputs, d * mul, 4, 2, 1))
            setattr(self, "conv%d_bn" % (i + 1), nn.BatchNorm2d(d * mul))
            inputs = d * mul
            mul *= 2

        self.d_max = inputs

        self.fc1 = nn.Linear(inputs * 4 * 4, zsize)
        self.fc2 = nn.Linear(inputs * 4 * 4, zsize)

        self.d1 = nn.Linear(zsize, inputs * 4 * 4)

        mul = inputs // d // 2

        for i in range(1, self.layer_count):
            setattr(self, "deconv%d" % (i + 1), nn.ConvTranspose2d(inputs, d * mul, 4, 2, 1))
            setattr(self, "deconv%d_bn" % (i + 1), nn.BatchNorm2d(d * mul))
            inputs = d * mul
            mul //= 2

        setattr(self, "deconv%d" % (self.layer_count + 1), nn.ConvTranspose2d(inputs, channels, 4, 2, 1))

    def encode(self, x):
        for i in range(self.layer_count):
            x = F.relu(getattr(self, "conv%d_bn" % (i + 1))(getattr(self, "conv%d" % (i + 1))(x)))

        x = x.view(x.shape[0], self.d_max * 4 * 4)
        h1 = self.fc1(x)
        h2 = self.fc2(x)
        return h1, h2

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, x):
        x = x.view(x.shape[0], self.zsize)
        x = self.d1(x)
        x = x.view(x.shape[0], self.d_max, 4, 4)
        #x = self.deconv1_bn(x)
        x = F.leaky_relu(x, 0.2)

        for i in range(1, self.layer_count):
            x = F.leaky_relu(getattr(self, "deconv%d_bn" % (i + 1))(getattr(self, "deconv%d" % (i + 1))(x)), 0.2)

        x = F.tanh(getattr(self, "deconv%d" % (self.layer_count + 1))(x))
        return x

    def forward(self, x):
        mu, logvar = self.encode(x)
        mu = mu.squeeze()
        logvar = logvar.squeeze()
        z = self.reparameterize(mu, logvar)
        return self.decode(z.view(-1, self.zsize, 1, 1)), mu, logvar

    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)


def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

In [3]:
model_path = '/home/sriyar/ADRL_Assignment2/VAEmodel_butterfly.pkl'
vae = VAE(zsize=512, layer_count=5, channels=3)

vae.load_state_dict(torch.load(model_path))


vae.eval()  # Set the model to evaluation mode


/tmp/ipykernel_264101/851428903.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae.load_state_dict(torch.load(model_path))


VAE(
  (conv1): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv1_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3_bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4_bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(1024, 2048, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv5_bn): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=32768, out_features=512, bias=True)
  (fc2): Linear(in_features=32768, out_features=512, bias=

In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from datasets import load_dataset
from PIL import Image
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# File paths
dataset_filepath = r"/home/sriyar/ADRL_Assignment2/butterflies_data/2/Training_set.csv"
dataset_filepath_test = r"/home/sriyar/ADRL_Assignment2/butterflies_data/2/Testing_set.csv"

# Define image transformations
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load datasets
dataset_train = load_dataset('csv', data_files=dataset_filepath)

# Create a mapping from labels to indices
label_to_idx = {label: idx for idx, label in enumerate(set(dataset_train['train']['label']))}

# Function to transform images and labels for training dataset
def transform_image_train(data):
    image = Image.open('/home/sriyar/ADRL_Assignment2/butterflies_data/2/train/' + data['filename'])
    data['image'] = data_transforms(image)
    data['label'] = label_to_idx[data['label']]  # Convert string label to index
    return data

# Apply transformations to the datasets
dataset_train = dataset_train.map(transform_image_train)

# Set format for PyTorch tensors
dataset_train.set_format(type='torch', columns=['image', 'label'])

# Split the training dataset into train, validation, and test sets
train_size = int(1 * len(dataset_train['train']))  
val_size = int(0.0 * len(dataset_train['train']))   
test_size = len(dataset_train['train']) - train_size - val_size  
train_dataset, val_dataset, test_dataset = random_split(dataset_train['train'], [train_size, val_size, test_size])

# Create DataLoaders
dataloader_butterfly_train = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=5)
dataloader_butterfly_val = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=5)
dataloader_butterfly_test = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=5)

# Count unique classes in the training dataset
num_classes = len(label_to_idx)  # Unique class count from the mapping
print(f"Number of classes: {num_classes}")


Map:   0%|          | 0/6499 [00:00<?, ? examples/s]

Number of classes: 75


In [5]:

latent_vectors = []
original_labels=[]
with torch.no_grad():
    for batch in dataloader_butterfly_train:
        images=batch['image']
        labels=batch['label']
        images = images.to(device)
        labels=labels.to(device)
        vae.to(device)
        mu, logvar = vae.encode(images)
        latent_vectors.append(mu)
        original_labels.append(labels)

latent_vectors = torch.cat(latent_vectors)
original_labels = torch.cat(original_labels)
print(latent_vectors.shape)
print((original_labels.shape))

torch.Size([6499, 512])
torch.Size([6499])


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score



# Assuming you have the complete dataset in 'latent_vectors' and 'original_labels'
# Convert tensors to NumPy arrays
X = latent_vectors.cpu().numpy()  # Features
y = original_labels.cpu().numpy()  # Labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Evaluate the classifier
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Classification accuracy: {100*accuracy:.2f}")



Classification accuracy: 16.92


In [7]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'hidden_layer_sizes': [(128, 64), (256,), (128, 128, 64)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01],
}

# Create the MLPClassifier instance
mlp = MLPClassifier(max_iter=500, random_state=42)

# Instantiate the grid search
grid_search = GridSearchCV(mlp, param_grid, cv=5)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

# Evaluate the best model
best_mlp = grid_search.best_estimator_
y_pred = best_mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Classification accuracy: {accuracy:.2f}")


Best parameters found:  {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (256,), 'learning_rate_init': 0.001}
Classification accuracy: 0.21


4.Beta VAE

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from datasets import load_dataset
from PIL import Image
from torchvision.utils import make_grid
from torch.utils.tensorboard import SummaryWriter
import numpy as np


In [2]:
batch_size=64
latent_dim=100

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_filepath=r"/data1/Code/Nidhi/ADRLass2/Training_set.csv" 

data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),  
    transforms.ToTensor()            
])

dataset_train = load_dataset('csv', data_files=dataset_filepath)

def transform_image_train(data):
    image = Image.open('/data1/Code/Nidhi/ADRLass2/train_butterfly/'+data['filename'])
    data['image'] = data_transforms(image)
    return data
def transform_image_test(data):
    image = Image.open('/home/sriyar/ADRL_Assignment2/butterflies_data/2/test/'+data['filename'])
    data['image'] = data_transforms(image)
    return data
dataset_train = dataset_train.map(transform_image_train)
dataset_train.set_format(type='torch', columns=['image'])
print(dataset_train)
dataloader_butterfly_train = DataLoader(dataset_train['train'], batch_size=batch_size, shuffle=True, num_workers=5)


DatasetDict({
    train: Dataset({
        features: ['filename', 'label', 'image'],
        num_rows: 6499
    })
})


In [ ]:
# Define BetaVAE class
class ConvBetaVAE(nn.Module):
    def __init__(self, latent_dim):
        super(ConvBetaVAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),  # (B, 32, 64, 64)
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),  # (B, 64, 32, 32)
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # (B, 128, 16, 16)
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # (B, 256, 8, 8)
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(256 * 8 * 8, latent_dim)
        self.fc_logvar = nn.Linear(256 * 8 * 8, latent_dim)
        self.fc_decode = nn.Linear(latent_dim, 256 * 8 * 8)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # (B, 128, 16, 16)
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # (B, 64, 32, 32)
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),  # (B, 32, 64, 64)
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),  # (B, 3, 128, 128)
            nn.Sigmoid()  
        )

    def encode(self, x):
        x = self.encoder(x).view(x.size(0), -1)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        z = self.fc_decode(z).view(-1, 256, 8, 8)
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
#loss fucntion
def beta_vae_loss(recon_x, x, mu, logvar, beta):
    MSE = nn.functional.mse_loss(recon_x, x, reduction='sum')/batch_size
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE + beta * KLD

In [ ]:
#Training
beta_values = [0.01, 0.1, 1, 2]

def train_conv_beta_vae(beta, dataloader, num_epochs, writer):
    model = ConvBetaVAE(latent_dim=latent_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    epoch_losses = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch_idx, data in enumerate(dataloader):
            images = data['image'].to(device)
            optimizer.zero_grad()
            recon_images, mu, logvar = model(images)
            loss = beta_vae_loss(recon_images, images, mu, logvar, beta)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(dataloader)
        epoch_losses.append(avg_loss) 

        with torch.no_grad():
            z = torch.randn(batch_size, latent_dim).to(device)
            generated_images = model.decode(z).cpu()

            grid_generated = make_grid(generated_images, nrow=8, normalize=True)
            grid_original = make_grid(images.cpu(), nrow=8, normalize=True)P
            grid_reconstructed = make_grid(recon_images.cpu(), nrow=8, normalize=True)

            writer.add_image(f'Generated Images (beta={beta})', grid_generated, global_step=epoch)
            writer.add_image(f'Original Images (beta={beta})', grid_original, global_step=epoch)
            writer.add_image(f'Reconstructed Images (beta={beta})', grid_reconstructed, global_step=epoch)
        
        writer.add_scalar(f'Loss/beta_{beta}', avg_loss, epoch)
    # Save the model
    torch.save(model.state_dict(), f'/data1/Code/Nidhi/ADRLass2/beta_{beta}_vae_model.pth')

    with torch.no_grad():
        grid_original = make_grid(images, nrow=8, normalize=True).cpu()
        grid_reconstructed = make_grid(recon_images, nrow=8, normalize=True).cpu()
        grid_generated = make_grid(generated_images, nrow=8, normalize=True).cpu()

        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        ax.imshow(grid_original.permute(1, 2, 0))
        ax.axis('off')
        ax.set_title(f"Original Images (beta={beta})")
        plt.show()

        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        ax.imshow(grid_reconstructed.permute(1, 2, 0))
        ax.axis('off')
        ax.set_title(f"Reconstructed Images (beta={beta})")
        plt.show()

        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        ax.imshow(grid_generated.permute(1, 2, 0))
        ax.axis('off')
        ax.set_title(f"Generated Images (beta={beta})")
        plt.show()

    return epoch_losses 


